# Fiddler examples have moved! [Deprecation Notice]

Dear user thank you for using fiddler product, we appreciate your time! We have moved the examples to a new github repo located at the following link


***
# [New fiddler-examples repo](https://github.com/fiddler-labs/fiddler-examples)
***

# Fiddler-BigQuery Integration 2 - Event Publishing

This notebook is used in conjuction with the guide on how to use Fiddler for you machine learning model data stored in BigQuery. Link

In this notebook we will look at how we can use production data stored in BigQuery to publish to fiddler as production data for your ML model.

Link to [part 1](https://colab.research.google.com/github/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/integration-examples/bigquery/Fiddler-BigQuery%20Integration%20-%20Model%20Registration.ipynb) of the guide on how to upload baseline data and register model on Fiddler.

## Establish connection to BigQuery

Install required libraries to loading data from BigQuery

In [ ]:
# !pip install google-cloud
# !pip install google-cloud-bigquery[pandas]
# !pip install google-cloud-storage

Adding the location of the Google cloud auth key info as environment variable

In [ ]:
#Set environment variables for your notebook
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '<link to json key>'

In [ ]:
#Imports google cloud client library and initiates BQ service
from google.cloud import bigquery
bigquery_client = bigquery.Client()

Below we use the query to load the required data from BigQuery table

In [ ]:
#Write Query on BQ
QUERY = """
SELECT * FROM `fiddler-bq.fiddler_test.churn_prediction_production` 
  """

In [ ]:
#Run the query and write result to a pandas data frame
Query_Results = bigquery_client.query(QUERY)
production_df = Query_Results.to_dataframe()

#View top few rows of result
print(production_df.shape)
production_df.head()

## Publish data to Fiddler

### Imports

We will publish the BigQuery data that we have just imported, on the Fiddler platform.

If you are new to Fiddler and want a detailed explanation on how to setup the fiddler environment please refer to our [Getting Started Guide](https://docs.fiddler.ai/pages/getting-started/product-tour/).

In [ ]:
!pip install -q fiddler-client;

import numpy as np
import pandas as pd
import fiddler as fdl

print(f"Running client version {fdl.__version__}")

### Connect to Fiddler

In [ ]:
URL = #
ORG_ID = #
AUTH_TOKEN = #

In [ ]:
# Initiate Fiddler client
client = fdl.FiddlerApi(
    url=URL,
    org_id=ORG_ID,
    auth_token=AUTH_TOKEN
)

Convert the pandas DataFrame into a dictionary.

In [ ]:
production_events = production_df.to_dict(orient='records')
production_events[0]

You can use the client's `publish_event` function to start pumping data into Fiddler!
  
*Just include:*
1. The event dictionary
2. A timestamp for when the event occurred

In [ ]:
from tqdm import tqdm

for event in tqdm(production_events):

    client.publish_event(
        project_id=PROJECT_ID,
        model_id=MODEL_ID,
        event=event,
        event_timestamp=event['timestamp']
    )

Now that we have events published to Fiddler, we can access the URL below to monitor the publish events

In [ ]:
print('/'.join([URL, 'projects', PROJECT_ID, 'models', MODEL_ID, 'monitor']))